In [110]:
# old query
from bs4 import BeautifulSoup
from requests_html import HTMLSession

requests = HTMLSession()             # instantiate the HTMLSession Object, this returns executed JS code hence return html is the same as that of the page
                                     # with the requests library, via requests.get(url), that was not the case. The query randomly returns either run or unrun html

In [12]:
# new html query

# using selenium to scrape dynamic website
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import requests_html
import pandas as pd
import os
from bs4 import BeautifulSoup 
from requests_html import HTMLSession
from time import sleep

In [15]:
driver.quit()   # serve as restart mechanism 

In [17]:
# define the browser and add the -- headless argument

chromedriver_path = os.getcwd() + '\\chromedriver\\chromedriver.exe'

opts = Options()
# opts.headless = True
opts.add_argument(' — headless')
driver = webdriver.Chrome(chromedriver_path, options=opts)

url_q1 = 'https://ph.hotels.com/search.do?resolved-location=COUNTRY%3A10233139%3AUNKNOWN%3AUNKNOWN&destination-'
url_q2 = 'id=10233139&q-destination=Philippines&q-check-in=2021-10-10&q-check-out=2021-10-11&q-rooms=1&q-room-0-adults=2&q-room-0-children=0'

url  = url_q1 + url_q2
driver.maximize_window()
driver.get(url)                   # open the url, the process is automated

# the webpage is dynamic, continously scrolling down adds 12 more hotel cards in the results-list
# in order to gather enough data, we recursively execute a scroll down script

# with init at i = 1 with 12 hotel cards, keep scrolling
i = 1
batches = 84
wait_time = 10

while i <= batches: 
    # this adds 12 more hotel cards per iteration, if i > 84, the script stops, that is we end up with 1008 ...
    # ... hotel cards
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight,)")    # scroll down to the bottom
    print('webpage currently has {0} hotel cards'.format(i*12))
    print('loading next batch')
    i += 1
    sleep(10)   # gives time for the second scrolldown, adjust according to net speed.
                # faster net merits smaller sleep times
        
print('done')

In [18]:
page = driver.page_source    # get html format of the webpage at 1008 hotel cards
soup = BeautifulSoup(page, 'html.parser')


In [19]:
# hotel_cards :: BeautifulSoup Object per hotel card in site, with tag li
hotel_cardsBS = soup.select_one('body .resp-section .inner-section .resp-row .resp-col .main-inner section .h-listing .listings')   # BeautifulSoup object of containing all cards

In [20]:
hotel_cards = hotel_cardsBS.find_all('li', recursive=False) # creates one BS obj per card from hotel card BeautifulSoup
print('we have {0} distinct hotels'.format(len(hotel_cards)))

we have 26 distinct hotels


<h3> <b> Inspecting one of the hotel cards </b> </h3>
Upon inspection the ff were idd.
<li> an a tag with further data given by the attribute href
<li> address: in _2oHhXM contains address with format: specific, town/city, zip, province, Philippines 
<li> amenities: in ?
<li> price: in ? #rate is one day for 2 persons
<li> h ref has it all, must go to href for number of rooms
<blockquote>
    <li> number of rooms in:
    <li> customer reviews in:
    <li> add landmarks, and getting around: in 
</blockquote>

In [21]:
# try getting information from 1 hotel card
# proof of concept
hotel_sample = hotel_cards[1]

In [22]:
hotel_name = hotel_sample['data-title']
hotel_name


'Conrad Manila'

In [163]:
rating = hotel_sample.find('strong',class_='guest-reviews-badge').get_text()
rating_decimal = float(''.join(i for i in filter(None,rating) if (i.isdigit()|(i=='.'))))

9.0

In [24]:
amenities     = hotel_sample.select('.hmvt8258-amenities')
amenities_col = [amenity.get_text() for amenity in amenities[0].select('li')]
amenities_col

['Pool', 'Free parking', 'Airport transfer', 'Spa', 'Gym', 'Restaurant']

In [25]:
address = hotel_sample.find('span', class_='address').get_text()
address       # can be split; address.split(', ') returns a list of loc, loc, mmo .... regardless start from last entry
              # ie get the index of Philippines and go back this gives us | address.split(', ')[-1], pa front pero pa back

'Seaside Boulevard, Corner of Coral Way, Mall of Asia Complex, Pasay, Manila, 1300, Philippines'

In [157]:
price = hotel_sample.find('a', class_='price-link').get_text()          # returns the likes of 'P6,221' 
price_int = int(''.join(i for i in filter(None,price) if i.isdigit()))  # filter returns list of all chars in price ['P',',','6','7','8'], loop over each element and 
                                                                        # create a new list satisfying only i.isdigit, join this into a string convert string into int
price_int

6561

In [29]:
href     = hotel_sample.find('a', class_='property-name-link')['href']
href_url = 'https://ph.hotels.com'+ href                                             # this is the link to the individual hotel information

# open the page and get the ff
# number of roms
# customer review
# landmarks

requests = HTMLSession()

page_href = requests.get(href_url)           
soup_href = BeautifulSoup(page_href.content, 'html.parser')

In [94]:
landMarks_q1 = 'body div.resp-section main.inner-section div#property-details '
landMarks_q2 = 'div#flexible-container-bottom div.whats-around-content-landmarks-transport' 
landmarks_transport = soup_href.select_one(landMarks_q1 + landMarks_q2)                        # gets to long if both queries are in one line

landmarks = landmarks_transport.find(class_='whats-around-content landmarks')
landmarks_col = [landmark.get_text() for landmark in landmarks.select('div.landmarks-expandable-wrapper ul.landmark-list li')]

transports = landmarks_transport.find(class_='whats-around-content transport')
transport_categories = [category['class'][0] for category in transports.select('ul')]                 # categegory['class'] returns a list, iregardless the first entry is the actual category

transpo = lambda cat: [station.get_text() for station in transports.select('ul.{0} li'.format(cat))]  # helper function gets the list of all transports under category cat, cat :: str
transports_col = {cat:transpo(cat) for cat in transport_categories}

In [95]:
landmarks_col

['In Barangay 76',
 'SM Mall of Asia - 8 min walk',
 'SMX Convention Center - 3 min walk',
 'Mall of Asia Arena - 6 min walk',
 'World Trade Center Manila - 33 min walk',
 'Cultural Center of the Philippines - 44 min walk',
 "Children's Museum (Museo Pambata) - 4.2 mi / 6.7 km",
 'Rizal Park - 4.4 mi / 7.1 km']

In [96]:
transports_col

{'airport': ['Ninoy Aquino Intl. Airport (MNL) - 20 min drive '],
 'train-station': ['Manila Buenidia Station - 6 min drive ',
  'Manila Vito Cruz Station - 7 min drive ',
  'Manila Paco Station - 7 min drive '],
 'shuttle': ['Airport shuttle (surcharge)']}

In [124]:
hSize_q1 = '#at-a-glance div.cont-wrap '          
hSize_q2 = 'div.fact-sheet-columns div.col-8-24 div.info-box ul li'   # there are two instances of ul li tags in html, select_one always select first instance which is the one we need in our case
hotel_size = soup_href.select_one(hSize_q1 + hSize_q2).get_text()     # this is a bs4 object containing which has a child containing num of rooms
                                                                      # concatenated strings for shorter query in line
    

In [127]:
room_count    = [int(roomCount) for roomCount in hotel_size.split() if roomCount.isdigit()][0]
room_count

347

In [ ]:
'''
Works up to this point

Note:
- Change the link, used different check in and out times since unavailble na yung old check in out query. Was probably due to covid adjustments
  Needs cleaner code
- The individual queries will be removed once code is finished :: was simply following top down formats
- Selenium sometimes load vanilla link, the one with unclear attribute names. Working solution, was to reload quit driver then reload.
- works up to this point added numeral formatting
- comments left
'''

In [16]:
'''
print(
'''
hotel name                  | 
address                     |
rating                      |
amneties                    |
number of rooms             |
price (2 person/ 1 day)     |
reviews                     |
landmarks                   |
'''
).format(
    hotel_name,
    rating,
    amneties,
    room_count,
    price,
    reviews,
    landmarks)
'''

SyntaxError: invalid syntax (<ipython-input-16-ecc34dba13a1>, line 4)